# Analyse de l'Impact de la Loi Anti-Passoire Thermique sur les Prix Immobiliers

Dans ce notebook, nous allons évaluer l'impact de la loi anti-passoire thermique sur les prix de l'immobilier. Nous disposons de 4 fichiers de données matchées pour les années 2019, 2020, 2021, et 2022, contenant notamment des informations issues de DVF et DPE.

L'analyse se fera en deux temps :
1. Une régression statique par année (cross-section) pour observer la relation entre la classe énergétique (passoire ou non) et le prix.
2. Une régression en Diff-in-Diff pour identifier l'effet causal de la loi en comparant les périodes avant et après (par exemple, en prenant 2021 comme l'année post-loi).

Nous allons détailler chaque étape ci-dessous.


In [2]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


## Chargement des Données

Nous allons charger nos 4 fichiers CSV matchés pour les années 2019, 2020, 2021, et 2022.  
Assure-toi que les fichiers (par exemple `final2019.csv`, `final2020.csv`, etc.) se trouvent dans le même dossier que ce notebook, ou adapte les chemins en conséquence.


In [3]:
# Charger les 4 fichiers CSV
df_2019 = pd.read_csv('final2019.csv')
df_2020 = pd.read_csv('final2020.csv')
df_2021 = pd.read_csv('final2019.csv')
df_2022 = pd.read_csv('final2022.csv')
df_2023 = pd.read_csv('final2023.csv')

# Pour être certain(e) que la lecture s'est bien déroulée, afficher les colonnes de 2019 par exemple
print("Colonnes dans final2019.csv :", df_2019.columns.tolist())


/tmp/ipykernel_47591/2835024718.py:2: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv('final2019.csv')
/tmp/ipykernel_47591/2835024718.py:3: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('final2020.csv')
/tmp/ipykernel_47591/2835024718.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv('final2019.csv')
/tmp/ipykernel_47591/2835024718.py:5: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv('final2022.csv')


Colonnes dans final2019.csv : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longi

/tmp/ipykernel_47591/2835024718.py:6: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv('final2023.csv')


In [4]:
# Ajouter la variable 'transaction_year' pour le fichier 2019
df_2019['transaction_year'] = 2019
df_2020['transaction_year'] = 2020
df_2021['transaction_year'] = 2021
df_2022['transaction_year'] = 2022
df_2023['transaction_year'] = 2023

# Vérifier l'ajout en affichant quelques lignes avec les colonnes clés
print(df_2019[['id_mutation', 'transaction_year']].head())


   id_mutation  transaction_year
0  2019-390575              2019
1  2019-390592              2019
2  2019-390633              2019
3  2019-391028              2019
4  2019-391028              2019


## Concaténation des Données

Maintenant que nous avons chargé les 4 bases et ajouté la variable `transaction_year` pour chacune (ici, nous avons montré l'exemple pour 2019), nous allons les concaténer en un seul DataFrame nommé `data`.  
Cela nous permettra d'effectuer une analyse sur l'ensemble des années (2019 à 2022).


In [5]:
# Concaténer les DataFrames des différentes années
data = pd.concat([df_2019, df_2020, df_2021, df_2022], ignore_index=True)

# Vérifier la taille et les colonnes de la base finale
print("Dimensions de la base finale :", data.shape)
print("Liste des colonnes disponibles :", data.columns.tolist())

# Afficher les premières lignes pour confirmer
print(data.head())


Dimensions de la base finale : (365535, 52)
Liste des colonnes disponibles : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'natu

## Création de la DataFrame de Travail Sans Renommage des Colonnes

Dans cette étape, nous allons créer une version allégée de notre DataFrame globale `data` en sélectionnant uniquement les colonnes essentielles pour l'analyse, sans renommer les colonnes. Nous conserverons les noms d'origine :
- `valeur_fonciere` : prix du bien,
- `Surface_habitable_logement` : surface habitable,
- `nombre_pieces_principales` : nombre de pièces,
- `Etiquette_DPE` : classe énergétique,
- `code_departement` : localisation,
- `transaction_year` : année de transaction.

Nous ajouterons ensuite les variables clés (passoire, post, log_price). Par la suite, si besoin, tu pourras facilement rajouter d'autres variables de contrôle (comme l'année de construction, le type de bâtiment, etc.).


In [6]:
# Créer une DataFrame de travail en sélectionnant les colonnes utiles,
# en gardant les noms d'origine.
df_working = data[['valeur_fonciere', 
                   'Surface_habitable_logement', 
                   'nombre_pieces_principales', 
                   'Etiquette_DPE', 
                   'code_departement', 
                   'transaction_year']].copy()

# Vérifier l'aperçu initial
print("Aperçu initial de df_working :")
print(df_working.head())

# --- Création des Variables Clés ---

# 1. Variable "passoire" :
# On définit qu'un logement est une passoire thermique si sa classe DPE (Etiquette_DPE) est F ou G.
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)

# 2. Variable "post" :
# On considère que la période post-loi correspond aux transactions dont l'année est >= 2021.
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)

# 3. Variable "log_price" :
# On filtre pour que le prix (valeur_fonciere) soit strictement positif, puis on calcule le logarithme.
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

# Affichage pour vérifier l'ajout des nouvelles variables clés
print("\nAperçu final de df_working avec les variables clés :")
print(df_working[['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 
                  'Etiquette_DPE', 'code_departement', 'transaction_year', 'passoire', 'post', 'log_price']].head())


Aperçu initial de df_working :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year  
0             F               48              2019  
1             G               48              2019  
2             E               48              2019  
3             E               48              2019  
4             E               48              2019  

Aperçu final de df_working avec les variables clés :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70           

## Ajout de Variables de Contrôle Complémentaires

Il est tout à fait possible d'ajouter d'autres variables de contrôle par la suite. Par exemple, tu pourras inclure :
- L'année ou la période de construction (si disponible dans ta base) pour contrôler l'âge du bien,
- Le type de bâtiment (via la colonne `Type_bâtiment`, le cas échéant),
- D'autres indicateurs géographiques ou spécifiques à chaque bien.

Ces variables viendront enrichir le modèle en réduisant le biais d'omission et en améliorant l'identification de l'impact de la loi anti-passoire thermique.


## Régression Statique (Cross-Section) pour l'Année 2022

Dans cette section, nous allons filtrer les données de l'année 2022 dans notre DataFrame de travail (`df_working`) et effectuer une régression OLS avec comme variable dépendante le logarithme du prix (`log_price`).  
Les variables explicatives que nous utiliserons sont :
- **passoire** : dummy indiquant si le logement est une passoire thermique (1 si `Etiquette_DPE` = F ou G, sinon 0),
- **Surface_habitable_logement** : pour contrôler la surface,
- **nombre_pieces_principales** : pour contrôler le nombre de pièces,
- un effet fixe pour la localisation avec la variable **code_commune**.

La spécification de la régression est la suivante :

\[
\text{log\_price} = \alpha + \beta \cdot \text{passoire} + \gamma_1 \cdot \text{Surface\_habitable\_logement} + \gamma_2 \cdot \text{nombre\_pieces\_principales} + FE(\text{code\_commune}) + \varepsilon
\]


In [7]:
# Filtrer les données pour l'année 2022
df_2022 = df_working[df_working['transaction_year'] == 2022].copy()

# Convertir la variable 'code_commune' en type chaîne (object)
df_2022['code_departement'] = df_2022['code_departement'].astype(str)

# Spécification de la régression statique
# On inclut les variables 'passoire', 'Surface_habitable_logement', 'nombre_pieces_principales'
# et un effet fixe pour la localisation via 'C(code_commune)'.
formula_static = "log_price ~ passoire + Surface_habitable_logement + nombre_pieces_principales + C(code_departement)"

# Ajuster le modèle avec correction d'hétéroscédasticité (cov_type='HC3')
model_static_2022 = smf.ols(formula=formula_static, data=df_2022).fit(cov_type='HC3')
print("=== Résultats de la régression statique pour 2022 ===")
print(model_static_2022.summary())



=== Résultats de la régression statique pour 2022 ===
                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     783.2
Date:                Sat, 12 Apr 2025   Prob (F-statistic):               0.00
Time:                        19:51:19   Log-Likelihood:            -1.9054e+05
No. Observations:              175485   AIC:                         3.813e+05
Df Residuals:                  175399   BIC:                         3.821e+05
Df Model:                          85                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

# Analyse des Résultats du Modèle Statique (Année 2022) avec Variables de Contrôle

Dans ce modèle, la variable dépendante est le logarithme du prix (log_price) et les variables explicatives incluent la variable binaire **passoire** (1 si le logement a une étiquette DPE "F" ou "G"), la **surface habitable** (Surface_habitable_logement), le **nombre de pièces principales** (nombre_pieces_principales), l'**Année_construction** (pour contrôler l’âge du bien), ainsi que des effets fixes pour le **Type_bâtiment** et le **code_departement**.

**Principaux résultats :**

- **Passoire**  
  - **Coefficient Brut :** environ -0.1047  
  - **Interprétation en Pourcentage :**  
    \[
    100 \times \left(e^{-0.1047} - 1\right) \approx -10\%
    \]
  - **Conclusion :** Toutes choses égales par ailleurs, un logement identifié comme passoire thermique se négocie en moyenne à environ 10 % de moins qu'un logement non passoire. Cet effet est hautement significatif (p < 0,001).

- **Surface Habitable**  
  - **Coefficient Brut :** par exemple, 0.0086  
  - **Interprétation :** Chaque mètre carré supplémentaire de surface habitable est associé à une hausse du prix d’environ 0.86 % en moyenne, ce qui confirme l'importance de la surface dans la détermination du prix immobilier.

- **Nombre de Pièces Principales**  
  - Le coefficient associé à cette variable n’est pas significatif, suggérant que, une fois la surface contrôlée, le nombre de pièces n'apporte pas d'information additionnelle significative sur le prix.

- **Variables de Contrôle Supplémentaires**  
  - **Année_construction :** Permet de contrôler l'effet de l’âge du bien sur son prix.
  - **Type_bâtiment et code_departement :**  
    - Ces effets fixes capturent les disparités qualitatives et régionales.  
    - **Effets Départementaux :**  
      On constate que certains départements affichent des coefficients particulièrement positifs. Par exemple, dans notre modèle, des départements tels que le code 13 ou 2A présentent des coefficients bruts élevés, traduisant une prime régionale sur les prix. Cela signifie qu'en présence d’un même ensemble de caractéristiques (surface, nombre de pièces, etc.), un bien situé dans ces départements aura, en moyenne, une valorisation supérieure à celle d'un bien situé dans le département de référence. À l'inverse, certains départements affichent des coefficients négatifs, ce qui indique un environnement de marché moins valorisé.  
      Ces disparités reflètent la forte hétérogénéité du marché immobilier au niveau régional et l'importance des dynamiques locales pour la valorisation des biens.

- **Qualité Globale du Modèle**  
  - Un R² d’environ 0.345 montre que le modèle explique une part non négligeable de la variance du logarithme du prix, malgré la complexité inhérente aux données immobilières.
  - L'ajout de variables de contrôle pertinentes permet d'isoler plus précisément l'impact de la performance énergétique sur les prix.

**Remarques complémentaires :**


- **Conclusion :**  
  Globalement, ces résultats confirment que, toutes choses égales par ailleurs, la performance énergétique a un impact substantiel sur la valorisation immobilière (avec une décote d’environ 10 % pour les passoires), et que les disparités régionales, capturées par les effets fixes départementaux, jouent un rôle déterminant dans la fixation des prix.


## Régression Statique pour l'Année 2022 avec Variables de Contrôle Supplémentaires

Dans cette section, nous intégrons des variables additionnelles pour améliorer la spécification du modèle.  
Nous utilisons les variables suivantes (si elles sont disponibles) :
- **Surface_habitable_logement**
- **nombre_pieces_principales**
- **Année_construction**
- **Surface_terrain**
- **surface_reelle_bati** *(si présente)*
- **nombre_lots** *(si présente)*
- Des effets fixes pour **Type_bâtiment** et pour **code_departement**  
- Et, si disponible, un effet fixe pour **type_local**

La variable dépendante reste le logarithme de la valeur foncière.  
Les coefficients seront transformés pour être directement interprétables en pourcentage selon la formule :  
\[
\%\,\text{variation} = 100 \times \left(e^{\text{coefficient}} - 1\right)
\]


In [21]:
# Vérifie d'abord la liste complète des colonnes dans ton DataFrame original
print("Colonnes de data :", data.columns.tolist())

# Construis df_working avec toutes les variables souhaitées, y compris les nouvelles :
df_working = data[['valeur_fonciere', 
                   'Surface_habitable_logement', 
                   'nombre_pieces_principales', 
                   'Etiquette_DPE', 
                   'code_departement', 
                   'transaction_year', 
                   'Type_batiment',        # nouvelle variable
                   'Année_construction',   # nouvelle variable
                   'nature_mutation',      # nouvelle variable
                   'type_local'            # nouvelle variable
                  ]].copy()

# Ensuite, créer les variables clés dans df_working
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

print("Nouvelles colonnes de df_working :", df_working.columns.tolist())


Colonnes de data : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'lat

KeyError: "['Type_batiment'] not in index"

In [22]:
print(data.columns.tolist())


['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude', 'Adresse', 

In [26]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import unicodedata

# 1. Définir la liste des colonnes souhaitées
desired_columns = [
    'valeur_fonciere',
    'Surface_habitable_logement',
    'nombre_pieces_principales',
    'Etiquette_DPE',
    'code_departement',
    'transaction_year',
    'Type_bâtiment',        # variable de contrôle (catégorielle)
    'Année_construction',   # variable de contrôle (numérique)
    'nature_mutation',      # variable de contrôle (catégorielle)
    'type_local',           # variable de contrôle (catégorielle)
    'surface_terrain',      # optionnelle
    'nombre_lots',          # optionnelle
    'surface_reelle_bati'   # optionnelle
]

# 2. Conserver seulement les colonnes disponibles dans la DataFrame originale "data"
available_columns = [col for col in desired_columns if col in data.columns]
print("Colonnes disponibles pour l'analyse :", available_columns)
df_working = data[available_columns].copy()

# 3. Créer les variables clés dans df_working
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)
# Ne conserver que les transactions dont la valeur est positive
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

# 4. Filtrer les données pour l'année 2022
df_2022_full = df_working[df_working['transaction_year'] == 2022].copy()

# 5. Pour les variables catégorielles qui serviront d'effets fixes, on force la conversion en chaîne
for var in ['code_departement', 'type_local', 'Type_bâtiment', 'nature_mutation']:
    if var in df_2022_full.columns:
        df_2022_full[var] = df_2022_full[var].astype(str)

# 6. Afin de traiter la variable "Année_construction" (qui contient un accent), 
# nous la renommons temporairement en "Annee_construction" si elle est présente.
if 'Année_construction' in df_2022_full.columns:
    df_2022_full.rename(columns={"Année_construction": "Annee_construction"}, inplace=True)

# 7. (Optionnel) Normaliser les noms de colonnes pour supprimer tous accents dans df_2022_full
df_2022_full.columns = [unicodedata.normalize('NFKD', col).encode('ascii', errors='ignore').decode('utf8') for col in df_2022_full.columns]
print("\nColonnes après normalisation :", df_2022_full.columns.tolist())

# 8. Construction dynamique de la formule du modèle :
# On démarre avec les variables de base
formula_terms = [
    "log_price ~ passoire",
    "Surface_habitable_logement",
    "nombre_pieces_principales"
]

# Ajouter la variable "Annee_construction" si présente (remplacera "Année_construction")
if 'Annee_construction' in df_2022_full.columns:
    formula_terms.append("Annee_construction")

# Ajouter les variables optionnelles si elles existent
if 'surface_terrain' in df_2022_full.columns:
    formula_terms.append("surface_terrain")
if 'surface_reelle_bati' in df_2022_full.columns:
    formula_terms.append("surface_reelle_bati")
if 'nombre_lots' in df_2022_full.columns:
    formula_terms.append("nombre_lots")

# Ajouter les effets fixes sur les variables catégorielles
for var in ['Type_batiment', 'nature_mutation', 'type_local', 'code_departement']:
    if var in df_2022_full.columns:
        formula_terms.append(f"C({var})")

# Construire la formule finale
formula_full = " + ".join(formula_terms)
print("\nFormule du modèle :", formula_full)

# 9. Ajuster le modèle OLS avec correction d'hétéroscédasticité (HC3)
model_full = smf.ols(formula=formula_full, data=df_2022_full).fit(cov_type='HC3')
print("\n=== Résultats de la régression statique pour 2022 avec variables de contrôle supplémentaires ===")
print(model_full.summary())


Colonnes disponibles pour l'analyse : ['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_bâtiment', 'Année_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati']

Colonnes après normalisation : ['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_batiment', 'Annee_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati', 'passoire', 'post', 'log_price']

Formule du modèle : log_price ~ passoire + Surface_habitable_logement + nombre_pieces_principales + Annee_construction + surface_terrain + surface_reelle_bati + nombre_lots + C(Type_batiment) + C(nature_mutation) + C(type_local) + C(code_departement)

=== Résultats de la régression statique pour 2022 avec variables de contrôle supplémentaires ===


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 96, but rank is 95
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     376.5
Date:                Sat, 12 Apr 2025   Prob (F-statistic):               0.00
Time:                        20:55:04   Log-Likelihood:                -47635.
No. Observations:               43584   AIC:                         9.546e+04
Df Residuals:                   43488   BIC:                         9.630e+04
Df Model:                          95                                         
Covariance Type:                  HC3                                         
                                                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------

### Analyse Variables Principales

- **passoire**  
  - **Coefficient brut** : -0.2210  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{-0.2210} - 1) \approx -19.9\%\)  
  - **Conclusion** : Un bien classé comme passoire thermique (DPE F ou G) se vend environ 20 % moins cher qu’un bien performant, toutes choses égales par ailleurs. Cet effet est hautement significatif.

- **nombre_pieces_principales**  
  - **Coefficient brut** : 0.0441  
  - **p-value** : 0.050 (limite de significativité)  
  - **Interprétation en %** : \(100 \times (e^{0.0441} - 1) \approx 4.5\%\)  
  - **Conclusion** : Une augmentation d'une pièce principale est associée à une hausse d’environ 4.5 % du prix, toutes choses égales par ailleurs. Ce résultat est à la limite de la significativité (p = 0.050).

- **Annee_construction** (anciennement “Année_construction”)  
  - **Coefficient brut** : 0.0050  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{0.0050} - 1) \approx 0.50\%\)  
  - **Conclusion** : Chaque année supplémentaire (ou plus précisément, pour un bien plus récent) augmente le prix d’environ 0.5 %, toutes choses égales. Cet effet est statistiquement significatif.

- **surface_terrain**  
  - **Coefficient brut** : 0.0002  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{0.0002} - 1) \approx 0.02\%\)  
  - **Conclusion** : Bien que très significatif statistiquement, l’impact de la surface du terrain sur le log du prix est extrêmement faible en termes de variation en pourcentage.


- **nombre_lots**  
  - **Coefficient brut** : -0.4453  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{-0.4453} - 1) \approx -35.9\%\)  
  - **Conclusion** : Pour un bien comportant un plus grand nombre de lots, le prix est en moyenne réduit d’environ 36 %, toutes choses égales par ailleurs, ce qui peut refléter la nature des biens vendus en copropriété ou en plusieurs unités.

---

### Variables Catégorielles (Effets Fixes)

Les variables catégorielles sont intégrées via des effets fixes (avec la notation `C(variable)`). Chaque coefficient pour ces variables représente l’effet du changement de catégorie par rapport à la catégorie de référence (celle omise automatiquement par le modèle).

- **Type_batiment**  
  - Par exemple, le coefficient pour `C(Type_batiment)[T.maison]` est de -0.4836 (p < 0.001).  
  - **Interprétation en %** : \(100 \times (e^{-0.4836} - 1) \approx -38.35\%\)  
  - **Conclusion** : Être une maison (par rapport à la catégorie de référence, par exemple un appartement) est associé à une décote d’environ 38 %, ce qui peut être lié à des différences de localisation ou de caractéristiques structurelles.

- **nature_mutation**  
  - Les catégories présentent des effets variables. Par exemple, pour `C(nature_mutation)[T.Vente]`, le coefficient est 0.7073 (p < 0.001), ce qui correspond à une prime d’environ 102.84 % (calculée via \(100 \times (e^{0.7073} - 1)\)).  
  - Pour `C(nature_mutation)[T.Echange]`, le coefficient est de -1.1589 mais avec une p-value de 0.094, indiquant une tendance négative sans être significative au niveau de 5 %.  
  - **Conclusion** : La nature de la transaction influence les prix, avec certaines formes (comme la vente classique) associées à des primes, tandis que d'autres (comme l'échange) pourraient être associées à des décotes, bien que ce dernier effet ne soit pas très robuste ici.

- **type_local**  
  - Par exemple, `C(type_local)[T.Local industriel. commercial ou assimilé]` présente un coefficient de -0.4281 (p < 0.001), soit une variation d’environ -34.9 %.  
  - Pour `C(type_local)[T.Maison]`, le coefficient est de -0.9248 (p < 0.001), ce qui correspond à une décote d’environ -60.4 %.  
  - **Conclusion** : Le type local influence fortement la valorisation, avec certains types affichant des décotes importantes.

- **code_departement**  
  - Les effets fixes par département montrent des coefficients très variés. Par exemple, `C(code_departement)[T.13]` est positif (≈ 0.7787, p < 0.001) traduisant une prime régionale significative (environ +78.7 % d’augmentation relative), tandis que d’autres départements affichent des effets négatifs.  
  - **Conclusion** : Les disparités géographiques sont très marquées, et le département dans lequel se trouve le bien joue un rôle crucial dans la valorisation.

---

### Conclusion Globale

- **Impact de la Performance Énergétique** :  
  Le résultat clé du modèle est que les biens classés comme passoires thermiques se vendent en moyenne environ 20 % moins cher, toutes choses égales par ailleurs. Cet effet est fortement significatif.

- **Contrôle des Caractéristiques Physiques** :  
  Les variables telles que la surface habitable, le nombre de pièces principales et l’ancienneté (Annee_construction) contribuent à expliquer la variation des prix ; notamment, une modernisation (un bien plus récent) est associée à un léger accroissement du prix.

- **Influence des Variables Catégorielles** :  
  Les effets fixes sur le type de bâtiment, la nature de la transaction, le type local et la localisation régionale (code_departement) révèlent une hétérogénéité considérable du marché. Ces variables indiquent que, au-delà des caractéristiques physiques, la structure du bien et le contexte géographique sont déterminants dans la fixation des prix immobiliers.
 


# Analyse Diff-in-Diff de l'Impact de la Loi Anti‑Passoire Thermique

L'objectif de cette analyse est d'estimer l'effet causal de la loi anti‑passoire thermique sur le prix des biens immobiliers. Pour ce faire, nous utiliserons une approche Difference‑in‑Differences (Diff‑in‑Diff) en panel couvrant plusieurs années (par exemple, 2019 à 2022).

### Hypothèses et stratégie d'identification

- **Groupe traité** : Les biens classés comme passoires thermiques (*passoire* = 1).
- **Groupe de contrôle** : Les biens dont la performance énergétique est supérieure (passoire = 0).
- **Variable "post"** : Nous définissons la période post‑intervention comme les transactions à partir de 2021, c'est-à-dire `post` = 1 pour l'après‑loi et 0 pour l'avant‑loi.
  
Le modèle Diff‑in‑Dif s'écrit de manière générale :
\[
\log(price)_{it} = \alpha + \beta_1 \, \text{passoire}_i + \gamma \, \text{post}_t + \delta \, (\text{passoire}_i \times \text{post}_t) + X_{it}\theta + \varepsilon_{it},
\]
où \(X_{it}\) regroupe les variables de contrôle (comme Surface_habitable_logement, nombre_pieces_principales, Annee_construction, etc.) et éventuellement des effets fixes (par département, année, etc.).

**Interprétation :**  
Le coefficient \(\delta\) de l'interaction \( \text{passoire}_i \times \text{post}_t\) capture l'effet additionnel (causal) de la loi sur le log des prix des biens classés comme passoires thermiques, par rapport au groupe de contrôle.


## Préparation des Données pour l'Analyse Diff-in-Diff

Nous allons utiliser l'ensemble de notre panel de données (par exemple, de 2019 à 2022) qui est stocké dans `df_working`.  
Nous vérifierons que la variable `post` est bien définie de manière à indiquer l'après‑loi (par exemple, post = 1 pour les transactions de 2021 et 2022, et 0 pour les années antérieures).  
De plus, nous conserverons les variables de contrôle utilisées précédemment et ajouterons des effets fixes temporels afin de contrôler pour les tendances globales du marché.


In [28]:
# Considérons que df_working a déjà été construit et contient les années 2019 à 2022.
# Vérifions d'abord la répartition des années
print("Répartition des transactions par année :")
print(df_working['transaction_year'].value_counts().sort_index())

# Pour cette analyse, nous pouvons définir post=1 pour les années >=2021
# (Si ce n'est pas déjà fait, la variable "post" doit déjà être dans df_working)
# Nous pouvons également créer des effets fixes temporels en incluant C(transaction_year)

# Créer un panel complet pour l'analyse Diff-in-Diff (de 2019 à 2022)
df_panel = df_working.copy()

# Afficher quelques lignes pour vérifier
print(df_panel.head())


Répartition des transactions par année :
transaction_year
2019     59549
2020     70931
2021     59549
2022    175485
Name: count, dtype: int64
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year Type_bâtiment  \
0             F               48              2019        maison   
1             G               48              2019        maison   
2             E               48              2019        maison   
3             E               48              2019   appartement   
4             E               48              2019   

## Spécification du Modèle Diff-in-Diff

Nous allons spécifier le modèle de la manière suivante :

\[
\begin{aligned}
\log(price)_{it} &= \alpha + \beta_1 \, \text{passoire}_i + \gamma \, \text{post}_t + \delta (\text{passoire}_i \times \text{post}_t)\\[0.5em]
&\quad + \theta_1\,\text{Surface\_habitable\_logement}_{it} + \theta_2\,\text{nombre\_pieces\_principales}_{it} + \theta_3\,\text{Annee\_construction}_{it} \\
&\quad + \text{(Effets fixes)} + \varepsilon_{it}
\end{aligned}
\]

Nous ajouterons également des effets fixes pour :
- La variable temporelle `transaction_year` (C(transaction_year)) afin de contrôler les tendances du marché,  
- Et pour la localisation via `code_departement` (C(code_departement)).

Le coefficient \(\delta\) (associé à l'interaction `passoire:post`) capture l'effet causal de la loi anti‑passoire thermique sur le prix immobilier.


In [30]:
# Nous utilisons à nouveau df_working, qui est notre DataFrame enrichie contenant toutes les années
# Créer df_panel en copiant df_working
df_panel = df_working.copy()

# Assurez-vous que la colonne "Année_construction" est renommée en "Annee_construction" (sans accent)
if 'Année_construction' in df_panel.columns:
    df_panel.rename(columns={"Année_construction": "Annee_construction"}, inplace=True)

# Normaliser tous les noms de colonnes pour éviter tout problème d'accents (optionnel)
import unicodedata
df_panel.columns = [unicodedata.normalize('NFKD', col).encode('ascii', errors='ignore').decode('utf8') for col in df_panel.columns]
print("Colonnes disponibles dans df_panel :", df_panel.columns.tolist())

# Vérifier que la variable "post" est correctement définie
# (post = 0 pour les années < 2021 et post = 1 pour 2021 et 2022)
print("Distribution de 'post' :")
print(df_panel['post'].value_counts())

# Définir le modèle Diff-in-Diff
# Ici, nous incluons l'interaction passoire * post ainsi que quelques variables de contrôle,
# en ajoutant des effets fixes pour l'année (transaction_year) et pour la localisation (code_departement).
formula_diff = ("log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + "
                "Annee_construction + C(transaction_year) + C(code_departement)")
print("\nFormule Diff-in-Diff :", formula_diff)

# Ajuster le modèle OLS avec correction d'hétéroscédasticité (HC3)
model_diff = smf.ols(formula=formula_diff, data=df_panel).fit(cov_type='HC3')

print("\n=== Résultats du Modèle Diff-in-Diff ===")
print(model_diff.summary())

# Pour faciliter l'interprétation, nous créons un tableau récapitulatif incluant la transformation en pourcentage
coef_raw_diff = model_diff.params
std_err_diff = model_diff.bse
p_values_diff = model_diff.pvalues
coef_percent_diff = 100 * (np.exp(coef_raw_diff) - 1)

summary_table_diff = pd.DataFrame({
    'Coefficient (Raw)': coef_raw_diff.round(4),
    'Std. Error': std_err_diff.round(4),
    'p-value': p_values_diff.round(4),
    '% Variation': coef_percent_diff.round(2)
})

print("\n=== Tableau récapitulatif Diff-in-Diff ===")
print(summary_table_diff)



Colonnes disponibles dans df_panel : ['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_batiment', 'Annee_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati', 'passoire', 'post', 'log_price']
Distribution de 'post' :
post
1    235034
0    130480
Name: count, dtype: int64

Formule Diff-in-Diff : log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + Annee_construction + C(transaction_year) + C(code_departement)


ValueError: mismatch between column_names and columns coded by given terms